In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm

from keras.layers import Input
from keras.models import Model
from keras.callbacks import ModelCheckpoint

import data, img_utils, text_utils, model_utils

In [ ]:
CHARACTERS = "اآأدذجحخهعغفقثصضطكمنتلبيسشظزوةىلارؤءئ "
BASE_PATH = "..\..\OCR_fName_Task_Dataset"

In [ ]:
data_reader = data.ReadData(BASE_PATH)
image_obj = img_utils.ImageProcessingUtils()

In [ ]:
data_dict = data_reader.parse_data_from_folder()
print('Files Count:', len(data_dict.keys()))

In [ ]:
# To find text of max length
max_label_length = 0

#lists for train dataset
train_images = []
train_text = []
train_text_length = []
train_input_length = []
original_text = []


#lists for validation dataset
valid_images = []
valid_text = []
valid_input_length = []
valid_text_length = []
valid_original_text = []

## Loop on each file to read related image then start preprocessing on it

In [ ]:
i = 0
for key, value in tqdm(data_dict.items()):
    file_name = key + ".jpg"
    image_path = BASE_PATH + "\\" + file_name
    text = value
    image = image_obj.processing_pipeline(image_path)
    
    if image.shape != (32,128):
        print('resize error in file:', file_name)
        continue
    
    if len(text) > max_label_length:
        max_label_length = len(text)
    
    if i%10 != 0:
        original_text.append(text)
        train_images.append(image)
        train_text.append(text_utils.encode_to_labels(text, CHARACTERS))
        train_text_length.append(len(text))
        train_input_length.append(20)
    
    # 10% percent of data for validation
    else:
        valid_images.append(image)
        valid_text.append(text_utils.encode_to_labels(text, CHARACTERS))
        valid_text_length.append(len(text))
        valid_input_length.append(20)
        valid_original_text.append(text)
    i+=1

## Prepare Inputs For Model

In [ ]:
# Pad text
train_text_pad = text_utils.pad_text(train_text, max_label_length, pad_value=len(CHARACTERS))
valid_text_pad = text_utils.pad_text(valid_text, max_label_length, pad_value=len(CHARACTERS))

In [ ]:
# Convert Lists to Numpy Arrays
train_images = np.array(train_images)
train_input_length = np.array(train_input_length)
train_text_length = np.array(train_text_length)

valid_images = np.array(valid_images)
valid_input_length = np.array(valid_input_length)
valid_text_length = np.array(valid_text_length)

## Create CRNN & Add CTC Loss

In [ ]:
labels = Input(name='labels', shape=[max_label_length], dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')
filepath=".\\content\\best_model_v2.hdf5"

In [ ]:
inference_model, inputs, outputs = model_utils.create_model_architecture()
model = model_utils.add_ctc_layer(inputs, outputs, labels, input_length, label_length)
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer = 'adam', metrics='accuracy')
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss', verbose=1, save_best_only=True,
                             mode='auto', metrics=['accuracy'])
callbacks_list = [checkpoint]

## Model training for 75 epochs

In [ ]:
batch_size = 512
epochs = 75 
model.fit(x=[train_images, train_text_pad, train_input_length, train_text_length],
          y=np.zeros(len(train_images)), 
          batch_size=batch_size, epochs = epochs, 
          validation_data = ([valid_images, valid_text_pad, valid_input_length, valid_text_length], 
          [np.zeros(len(valid_images))]), verbose = 1, callbacks = callbacks_list)